# Google Drive connect

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#yolov8 훈련하기 시작

In [ ]:
%mkdir dataset # Model 저장 데이터 생성

## 집파일 경로 복사해서 하단 경로에 적용

In [ ]:
!cp /content/drive/MyDrive/human_data.zip /content/ # Google Drive 안에 있는 .zip 파일을 불러와 코랩 드라이브에 복사해서 업로드

In [ ]:
!unzip /content/human_data.zip -d dataset/  # 복사한 .zip 파일을 dataset으로 압축해제

## 압축해제가 잘 됐는지 /dataset/test 파일 내 아무 이미지 갖고 와서 확인

In [ ]:
# Check if image all loarded ok
import glob
import cv2
from google.colab.patches import cv2_imshow
import random


# 이미지 파일 경로를 가져오기
image_paths = glob.glob('/content/dataset/test/images/*.jpg')

# 이미지가 없는 경우 에러 처리
if not image_paths:
    print("이미지 파일이 없습니다.")
else:
    # 랜덤하게 이미지 선택
    selected_image_path = random.choice(image_paths)

    # 이미지 표시
    img = cv2.imread(selected_image_path)
    print(img.shape)
    cv2_imshow(img)

## .yaml 파일 생성
### **labelImg에서 정한 클래스 이름으로 맞춰주기, 대소문자 주의!!!!!**

새로운 yaml파일을 만들고 train, val, test 데이터셋 경로가 올바른게 작성해줘야 합니다



```
변경전                       변경후
train: ../train/images  ->  'train' : '/content/dataset/train/images'
```

In [ ]:
import yaml

data = {
        # 'train' : '/content/dataset/my_data/train/images',
        # 'test' : '/content/dataset/my_data/test/images',
        # 'val' : '/content/dataset/my_data/valid/images',
        'train' : '/content/dataset/train/images',
        'test' : '/content/dataset/test/images',
        'val' : '/content/dataset/valid/images',

        'nc': 1,
        # 'names': ['car'] #### labelImg에서 정한 클래스 이름으로 맞춰주기, 대소문자 주의!!!!!
        'names': ['human']

        # 'names': ['Truck','Dummy']
        # 'names': ['Normal','Error']
        # 'names' : ['NG', 'OK']
}

# with open('/content/dataset/my_data/custom_data.yaml', 'w') as f:
with open('/content/dataset/custom_data.yaml', 'w') as f:
  yaml.dump(data, f)

custom_data.yaml 파일을 만들었으니 제대로 작성이 되었는지 확인합니다

In [ ]:
# with open('/content/dataset/my_data/custom_data.yaml', 'r') as f:
with open('/content/dataset/custom_data.yaml', 'r') as f:
  view_yaml = yaml.safe_load(f)
  display(view_yaml)


## Yolo8n install

In [ ]:
!pip install ultralytics

### Yolo install Check!!!

In [ ]:
import ultralytics

ultralytics.checks()

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

In [ ]:
print(type(model.names),len(model.names))

print(model.names)

In [ ]:
results = model.predict('https://ultralytics.com/images/bus.jpg', save=True, show=True)

In [ ]:
from IPython.display import Image, clear_output
Image(filename='/content/runs/detect/predict/bus.jpg',width=1000)

In [ ]:
# model.train(data='/content/dataset/my_data/custom_data.yaml', epochs= 100, patience = 20, batch=32, imgsz =640)
model.train(data='/content/dataset/custom_data.yaml', epochs= 100, patience = 20, batch=32, imgsz =640)

### F1 score calculate

f1 = 2*(precision * recall) / (precision + 1)

In [ ]:
f1 = 2*0.9940479577432576*1 / (0.9940479577432576 + 1)
f1

In [ ]:
# model.train(data='/content/dataset/my_data/custom_data.yaml', epochs= 100, patience = 20, batch=32, imgsz =640)
model.train(data='/content/dataset/custom_data.yaml', epochs= 100, patience = 20, batch=32, imgsz =640)

In [ ]:
from IPython.display import Image, clear_output
Image(filename='/content/runs/detect/train/results.png',width=1000)

### F1-Score Cal
<bar>
ex) <bar>
Results saved to runs/detect/val2
모델 검증이 완료되었습니다. <bar>

Please check the directory: runs/detect/val2 for the F1-Confidence curve plot (usually named 'F1_curve.png' or similar). <bar>

**저장된 모델 이름 확인, 경로 따라가면 계산된 결과 이미지 파일이 존재함**

In [ ]:
# 1. Ultralytics 라이브러리 설치
!pip install ultralytics -q

import torch
from ultralytics import YOLO
import numpy as np
import matplotlib.pyplot as plt
import yaml

# 2. YOLOv8n 모델 로드
# 만약 직접 학습시킨 모델 가중치 파일(.pt)이 있다면 경로를 수정해주세요.
model = YOLO('/content/runs/detect/train/weights/best.pt')


# 3. 모델 검증(Validation) 실행
# 'data' 인자에는 직접 사용하실 데이터셋의 .yaml 파일 경로를 지정해야 합니다.
# plots=True를 주면 F1-Confidence 커브를 포함한 다양한 결과가 runs/detect/val/ 폴더에 저장됩니다.
print("모델 검증을 시작합니다. 데이터셋 크기에 따라 시간이 소요될 수 있습니다...")

# Load your custom data.yaml file
custom_data_path = '/content/dataset/custom_data.yaml'
with open(custom_data_path, 'r') as f:
    data_yaml = yaml.safe_load(f)

validator = model.val(data=custom_data_path, plots=True)
print("모델 검증이 완료되었습니다.")

# The F1-Confidence curve plot should be saved in the runs/detect/val/ directory
print(f"Please check the directory: {validator.save_dir} for the F1-Confidence curve plot (usually named 'F1_curve.png' or similar).")

# Note: Directly extracting F1 scores and confidence thresholds for plotting
# from the validator object is not straightforward in this version.
# Relying on the built-in plotting with plots=True is the recommended approach.

### Dumy

### train 사진 갖고와서 결과 확인, train의 라벨 갖고와서 감지하는지 확인

In [ ]:
Image(filename='/content/runs/detect/train/train_batch0.jpg',width=1000)
# Image(filename='/content/runs/detect/train3/train_batch0.jpg',width=1000)

In [ ]:
Image(filename='/content/runs/detect/train/val_batch0_labels.jpg',width=1000)

### 모델 클래스 이름 불러오기, 클래스 이름 확인용

In [ ]:
print(type(model.names),len(model.names))

print(model.names)

## train 내의 이미지, 라벨 함께 출력

In [ ]:
Image(filename='/content/runs/detect/train/val_batch0_pred.jpg',width=1000)

## 결과물 출력 부분
<bar>
학습된 결과물을 출력하는 부분입니다

마지막 출력문인 'Results saved to '이후에 나오는 경로가 학습된 이미지 출력이 있는 폴더의 경로입니다

In [ ]:
model = YOLO('/content/runs/detect/train/weights/best.pt')

In [ ]:
print(type(model.names),len(model.names))

print(model.names)

In [ ]:
# 테스트 이미지

from glob import glob

test_image_list = glob('/content/dataset/test/images/*')

print(len(test_image_list))

test_image_list.sort()

for i in range(len(test_image_list)):

    print('i = ',i, test_image_list[i])

In [ ]:
# results = model.predict(source ='/content/dataset/my_data/test/images/', save=True)
results = model.predict(source ='/content/dataset/test/images/', save=True)

## 해당 경로에서 랜덤으로 이미지 출력을 보는 코드입니다

colab은 특수한 패키지를 사용해야 볼 수 있습니다

In [ ]:
# 이미지 파일 경로를 가져오기
# image_paths = glob('/content/runs/detect/predict2/*.jpg')
image_paths = glob('/content/runs/detect/predict2/*.jpg')

# 이미지가 없는 경우 에러 처리
if not image_paths:
    print("이미지 파일이 없습니다.")
else:
    # 랜덤하게 이미지 선택
    # selected_image_path = random.choice(image_paths)
    # 이미지 표시
    # img = cv2.imread(selected_image_path)

    for image_path in image_paths:
      img = cv2.imread(image_path)
      cv2_imshow(img)

# Model Save

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

## 코랩 경로 맞춰줘야 함 , 경로 주의!!!!!

In [ ]:
!cp /content/runs/detect/train/weights/best.pt /content/drive/MyDrive/my_best_human.pt # 앞은 생성된 best.pt 파일, 뒤는 드라이브에 저장할 파일 / 이름 설정 필요